In [10]:
from pyro.infer import SVI, infer_discrete, TraceEnum_ELBO, TraceGraph_ELBO
import anneal.core
from anneal.models.MixtureGaussian import MixtureGaussian
from pyro.optim import ClippedAdam
import anneal.utils 

from importlib import reload  

reload(anneal.core)
reload(anneal.utils)

interface = anneal.core.Interface()
interface.set_model(MixtureGaussian)
interface.set_loss(TraceEnum_ELBO)
interface.set_optimizer(ClippedAdam)

data_dict = load_simulation_seg("example1", "example1")
data_dict

{'data': tensor([[45., 36., 42.,  ..., 50., 47., 42.],
         [55., 55., 60.,  ..., 76., 88., 89.],
         [32., 14., 27.,  ..., 34., 32., 42.],
         ...,
         [43., 33., 43.,  ..., 53., 53., 51.],
         [13.,  3., 12.,  ..., 21., 19.,  9.],
         [28., 21., 40.,  ..., 45., 50., 46.]]),
 'pld': tensor([2.0000, 2.0000, 1.0000, 2.0000, 2.0000, 2.0000, 2.0000, 1.3000, 2.0000,
         2.0000, 1.0000, 1.7000, 2.0000, 2.0000, 3.0000, 2.0000, 2.0000, 2.0000,
         2.0000, 3.0000, 1.0000, 2.0000, 2.0000, 2.0000, 2.0000, 2.0000, 2.0000,
         2.0000, 2.0000, 2.0000, 1.0000, 2.0000]),
 'segments': 32,
 'mu': tensor([12, 17, 14, 13,  8, 12, 29, 11, 12, 24, 15, 15, 12, 15, 11,  9, 13,  9,
         12, 26, 20, 17, 16,  8,  8, 15, 16, 13,  9, 11,  7,  9])}

In [11]:
pyro.enable_validation(True)
interface.initialize_model(data_dict)

loss = interface.run(300, MAP = True, param_optimizer={"lr" : 0.05 }, verbose=False)

Running MixtureGaussian on 1000 cells for 300 steps
..........

..........

..........

Done!


In [12]:
lr = interface.learned_parameters(posterior=True, verbose=True)

Computing assignment probabilities
    0	116.2011953125
    1	115.9626796875
    2	115.72515625
    3	115.4888984375
    4	115.2542265625
    5	115.0214296875
    6	114.790828125
    7	114.5626640625
    8	114.33725
    9	114.1148125
   10	113.8956484375
   11	113.6799375
   12	113.4679140625
   13	113.2597734375
   14	113.05571875
   15	112.8559140625
   16	112.6605
   17	112.4696171875
   18	112.2833828125
   19	112.1019296875
   20	111.925296875
   21	111.75359375
   22	111.586890625
   23	111.425171875
   24	111.2685
   25	111.116921875
   26	110.9703828125
   27	110.8288984375
   28	110.6924140625
   29	110.5608828125
   30	110.434328125
   31	110.31259375
   32	110.195640625
   33	110.0833671875
   34	109.975703125
   35	109.8725234375
   36	109.77371875
   37	109.679171875
   38	109.58875
   39	109.50234375
   40	109.4197890625
   41	109.341
   42	109.2657890625
   43	109.1940625
   44	109.1256484375
   45	109.0604609375
   46	108.9983046875
   47	108.9390703125
   48	108.882617

In [15]:
reload(anneal.utils)


anneal.utils.write_results(lr, "bau", True, "Bau")

Directory already existing, saving there
